# Create a Bucket and Lock it Down to a Specific VPC

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
s3 = boto3.Session().client(service_name='s3', region_name=region)
ec2 = boto3.Session().client('ec2')


In [ ]:
import time

timestamp = int(time.time())

bucket_secure_name = 'bucket-secure-bucket-policy-vpc-{}'.format(timestamp)
print(bucket_secure_name)

In [ ]:
!aws s3 mb s3://$bucket_secure_name

In [ ]:
!aws s3 ls s3://$bucket_secure_name

In [ ]:
vpc_id = ec2.describe_vpcs()['Vpcs'][0]['VpcId']

In [ ]:
# Create the bucket policy
bucket_policy_deny = {
        "Version": "2008-10-17",
        "Statement": [
            {
                "Effect": "Deny",
                "Principal": "*",
                "Action": [
                    "s3:ListBucket"
                ],
                "Resource": [
                    "arn:aws:s3:::{}".format(bucket_secure_name)
                ],
                "Condition": {
                    "StringNotEquals": {
                        "aws:sourceVpce": vpc_id
                    }
                }
            }
        ]
    }

# Set the Deny Policy on the Bucket

In [ ]:
import json

s3.put_bucket_policy(Bucket=bucket_secure_name, Policy=json.dumps(bucket_policy_deny))

# Verify `AccessDenied`

In [ ]:
!aws s3 ls s3://$bucket_secure_name